Loading necessary libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from tqdm import tqdm
import pandas  as pd
import xgboost as xgb

In [2]:
train_data = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test_data  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
train_data.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [3]:
features = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']

X_train = train_data[features]
y_train = train_data["target"]
final_X_test = test_data[features]

Splitting the training data

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1)

# Training XGBoostRegressor

Parameters adapted from this great notebook, 
https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna

In [5]:
Best_trial = {'lambda': 0.0030282073258141168, 'alpha': 0.01563845128469084, 'colsample_bytree': 0.5,
             'subsample': 0.7,'n_estimators': 4000, 'learning_rate': 0.01,'max_depth': 15,
             'random_state': 2020, 'min_child_weight': 257,'tree_method':'gpu_hist'
             ,'predictor': 'gpu_predictor'}

In [6]:
regressor = xgb.XGBRegressor(**Best_trial)

regressor.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)],verbose=False)

XGBRegressor(alpha=0.01563845128469084, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', lambda=0.0030282073258141168,
             learning_rate=0.01, max_delta_step=0, max_depth=15,
             min_child_weight=257, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=4000, n_jobs=0, num_parallel_tree=1,
             predictor='gpu_predictor', random_state=2020,
             reg_alpha=0.015638452, reg_lambda=0.00302820723,
             scale_pos_weight=1, subsample=0.7, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

In [7]:
y_xgb_pred = regressor.predict(final_X_test)

In [8]:
output1 = pd.DataFrame({"id":test_data.id, "target":y_xgb_pred})
output1.to_csv('xgb.csv', index=False)

# LightGBM Model

parameters from this awesome kernal 
https://www.kaggle.com/hamditarek/tabular-playground-series-xgboost-lightgbm

In [9]:
params ={'random_state': 33,'n_estimators':5000,
 'min_data_per_group': 5,
 'boosting_type': 'gbdt',
 'device_type' : 'gpu',
 'num_leaves': 256,
 'num_iterations' : 5000,
 'max_dept': -1,
 'learning_rate': 0.005,
 'subsample_for_bin': 200000,
 'lambda_l1': 1.074622455507616e-05,
 'lambda_l2': 2.0521330798729704e-06,
 'n_jobs': -1,
 'cat_smooth': 1.0,
 'silent': True,
 'importance_type': 'split',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'bagging_fraction': 0.8206341150202605,
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.001,
 'bagging_freq': 6,
 'feature_fraction': 0.5,
 'min_gain_to_split': 0.0,
 'min_child_samples': 20}

In [10]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(**params)
lgb_model.fit(X_train, y_train, eval_set=(X_test, y_test),early_stopping_rounds = 50, verbose = 0)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMRegressor(bagging_fraction=0.8206341150202605, bagging_freq=6,
              cat_smooth=1.0, device_type='gpu', feature_fraction=0.5,
              feature_pre_filter=False, lambda_l1=1.074622455507616e-05,
              lambda_l2=2.0521330798729704e-06, learning_rate=0.005,
              max_dept=-1, metric='rmse', min_data_in_leaf=100,
              min_data_per_group=5, min_gain_to_split=0.0,
              min_sum_hessian_in_leaf=0.001, n_estimators=5000,
              num_iterations=5000, num_leaves=256, random_state=33)

In [11]:
y_pred_lgb = lgb_model.predict(final_X_test)

In [12]:
output2 = pd.DataFrame({"id":test_data.id, "target":y_pred_lgb})
output2.to_csv('lgbm.csv', index=False)

# CatBoost Model

Parameters from this awesome notebook
https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna#Catboost-Using-Optuna

In [13]:
Best_trial = {'l2_leaf_reg': 0.02247766515106271, 'max_bin': 364, 'subsample': 0.6708650091202213,
             'learning_rate': 0.010290546311954876, 'max_depth': 10, 'random_state': 24, 'min_data_in_leaf': 300,
            'loss_function': 'RMSE','n_estimators':  25000,'rsm':0.5}

In [14]:
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(**Best_trial)

In [15]:
cb_model.fit(X_train, y_train,eval_set=(X_test, y_test),use_best_model=True,verbose=1000)

0:	learn: 0.7327592	test: 0.7331505	best: 0.7331505 (0)	total: 171ms	remaining: 1h 11m 17s
1000:	learn: 0.6913845	test: 0.7052213	best: 0.7052213 (1000)	total: 1m 41s	remaining: 40m 41s
2000:	learn: 0.6742225	test: 0.7016200	best: 0.7016194 (1999)	total: 3m 23s	remaining: 38m 56s
3000:	learn: 0.6590669	test: 0.6998760	best: 0.6998760 (3000)	total: 5m 6s	remaining: 37m 24s
4000:	learn: 0.6451891	test: 0.6990137	best: 0.6990137 (4000)	total: 6m 47s	remaining: 35m 40s
5000:	learn: 0.6325288	test: 0.6983475	best: 0.6983474 (4999)	total: 8m 29s	remaining: 33m 59s
6000:	learn: 0.6203913	test: 0.6980776	best: 0.6980758 (5998)	total: 10m 13s	remaining: 32m 21s
7000:	learn: 0.6091375	test: 0.6980490	best: 0.6980152 (6805)	total: 11m 56s	remaining: 30m 40s
8000:	learn: 0.5986115	test: 0.6980575	best: 0.6979833 (7481)	total: 13m 40s	remaining: 29m 2s
9000:	learn: 0.5885702	test: 0.6981016	best: 0.6979833 (7481)	total: 15m 23s	remaining: 27m 21s
10000:	learn: 0.5789397	test: 0.6981202	best: 0.6979

In [16]:
cat_pred =cb_model.predict(final_X_test)

In [17]:
output3 = pd.DataFrame({"id":test_data.id, "target":cat_pred})
output3.to_csv('cat.csv', index=False)

In [18]:
results = (y_pred_lgb + y_xgb_pred + cat_pred) / 3 

output = pd.DataFrame({"id":test_data.id, "target":results})
output.to_csv('submission.csv', index=False)

In [19]:
output

,id,target
0,0,7.978564
1,2,7.888629
2,6,7.950746
3,7,8.302050
4,10,8.092851
...,...,...
199995,499984,8.169067
199996,499985,8.161149
199997,499987,8.211521
199998,499988,8.000527
